In [1]:
import json
from selenium import webdriver
import pandas as pd
import numpy as np
import re
import requests
import xml.etree.ElementTree as ET
from time import sleep

In [2]:
driver = webdriver.Chrome('C:\\Users\\Joanna\\AppData\\Local\\Programs\\Python\\Python39\\venv\\Scripts\\chromedriver.exe')
keywords = ['czerwona sukienka']
driver.get('https://www.google.pl/search?q=' + keywords[0])

In [3]:
webelements = driver.find_elements_by_xpath('//div[@class="rc"]/div/a')
urls = []
for elem in webelements:
    urls.append(elem.get_attribute('href'))
print(urls)

['https://domodi.pl/czerwone-sukienki', 'https://allani.pl/wyszukaj/sukienki/czerwone', 'https://www.zalando.pl/odziez-damska-sukienki/_czerwony/', 'https://allegro.pl/listing?string=czerwona%20sukienka', 'https://www.renee.pl/sukienki/czerwone', 'https://ohso.pl/c4558,4614,0,0,,czerwone-sukienki', 'https://modnakiecka.pl/pl/c/Czerwone-Sukienki/2240726', 'https://www.c-and-a.com/pl/pl/shop/trend-czerwona-sukienka', 'https://www.kartes-moda.pl/sukienki/czerwone']


In [4]:
headings1 = []
titles = []
bodies = []
for url in urls:
    driver.get(url)
    heading1 = driver.find_elements_by_tag_name('h1')
    headings1.append(heading1[0].text)
    title = driver.title
    titles.append(title)
    body = driver.find_elements_by_tag_name('body')
    bodies.append(body[0].text)

In [5]:
df = pd.DataFrame(
    {'site' : urls[0],
     'title': titles,
     'h1' : headings1,
     'content' : bodies
     })
df.head()

,site,title,h1,content
0,https://domodi.pl/czerwone-sukienki,"Czerwone sukienki, zima 2020 w Domodi","Czerwone sukienki, zima 2020",Skompletuj swoją szafę na nowy rok z rabatami ...
1,https://domodi.pl/czerwone-sukienki,"Czerwone sukienki, zima 2021","Czerwone sukienki, zima 2021",Damskie\nMęskie\nDziecięce\nUroda\nTrendy\nWyp...
2,https://domodi.pl/czerwone-sukienki,Czerwone Sukienki dla Ciebie! Odkryj tą idealn...,czerwone Sukienki,PRZEJDŹ DO GŁÓWNEJ ZAWARTOŚCI\nZimowa wyprzeda...
3,https://domodi.pl/czerwone-sukienki,Czerwona sukienka - Niska cena na Allegro.pl,szukaszczerwona sukienka,szukaj wielu\nWszędzie\nWszystkie kategorie\nD...
4,https://domodi.pl/czerwone-sukienki,Czerwone sukienki damskie: zima 2020 na renee.pl,Czerwone sukienki damskie,ROZPOCZNIJ NOWY ROK W DOBYM STYLU I ODBIERZ -3...


In [6]:
def collect_data(url, category):
    api_content = requests.get('https://pagespeedonline.googleapis.com/pagespeedonline/v5/runPagespeed?url={}&category={}'.format(url, category)).content
    content = json.loads(api_content)
#     print(content)
    
    values = ['id', 'score', 'title', 'description']
    records = []

    for elem in content['lighthouseResult']['categories'][category.replace('_','-')]['auditRefs']:
        record = [{'url' : url}]
        record = []
        record.append({'id': elem['id']})
        record.append({'weight' : elem['weight']})
        try:
            group = elem['group']
        except KeyError:
            group = np.nan
        record.append({'group' : group})
        records.append(record)
        values = ['score', 'title', 'description']
        for key, value in content['lighthouseResult']['audits'].items():
            if key == elem['id']:
                for v in values:
                    record.append({v : value[v]})
                    
    cols = ['id', 'weight', 'group', 'score', 'title', 'description']
    rows = []
    for record in records:
        row = []
        for elem in record:
            row.append([elem[key] for key in elem][0])
        rows.append(row)

    df = pd.DataFrame(rows, columns=cols )
    return df

In [7]:
# categories = [{'performance' : 'performance'}] #,{'best_practices' : 'best-practices'},{'SEO' :'SEO'}
categories = ['performance', 'best_practices', 'seo']
dfs = []
for url in urls:
    for category in categories:
        while True:
            try:
                df = collect_data(url, category)
            except KeyError:
                sleep(100)
                continue
            break
        dfs.append(df)

In [8]:
dfs[1]

,id,weight,group,score,title,description
0,is-on-https,1,best-practices-trust-safety,1.0,Uses HTTPS,"All sites should be protected with HTTPS, even..."
1,external-anchors-use-rel-noopener,1,best-practices-trust-safety,1.0,Links to cross-origin destinations are safe,"Add `rel=""noopener""` or `rel=""noreferrer""` to ..."
2,geolocation-on-start,1,best-practices-trust-safety,1.0,Avoids requesting the geolocation permission o...,Users are mistrustful of or confused by sites ...
3,notification-on-start,1,best-practices-trust-safety,1.0,Avoids requesting the notification permission ...,Users are mistrustful of or confused by sites ...
4,no-vulnerable-libraries,1,best-practices-trust-safety,0.0,Includes front-end JavaScript libraries with k...,Some third-party scripts may contain known sec...
5,password-inputs-can-be-pasted-into,1,best-practices-ux,1.0,Allows users to paste into password fields,Preventing password pasting undermines good se...
6,image-aspect-ratio,1,best-practices-ux,1.0,Displays images with correct aspect ratio,Image display dimensions should match natural ...
7,image-size-responsive,1,best-practices-ux,1.0,Serves images with appropriate resolution,Image natural dimensions should be proportiona...
8,preload-fonts,0,best-practices-ux,NaN,Fonts with `font-display: optional` are preloaded,Preload `optional` fonts so first-time visitor...
9,doctype,1,best-practices-browser-compat,1.0,Page has the HTML doctype,Specifying a doctype prevents the browser from...
